In [1]:
import h5py
import tensorflow as tf
import os
import numpy as np
import json
import shutil

2024-08-12 13:22:15.106484: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 13:22:15.133442: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 13:22:15.256626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 13:22:15.256676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 13:22:15.278317: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
num_episodes = 900
num_files = 150

input_hdf5_dir_1 = '/media/sbr-tech/Desk SSD1/aloha/datas_1of4_cucumbers_150'
input_hdf5_dir_2 = '/media/sbr-tech/Desk SSD1/aloha/datas_2of4_cucumbers_300'
input_hdf5_dir_3 = '/media/sbr-tech/Desk SSD1/aloha/datas_3of4_cucumbers_450'


input_hdf5_basename = "episode_{}.hdf5"

output_root = "/media/sbr-tech/Desk SSD1/aloha/"
dataset_name = "aloha_sim_multi_cucumber_dataset"
task_name = "aloha_sim_multi_cucumber_scripted_dataset"
version = "1.0.0"

output_basename = task_name + "-train.tfrecord-{}-of-{}"

In [3]:
output_dir = os.path.join(output_root, dataset_name, task_name, version)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
episodes_per_file = num_episodes // num_files

with open("dataset_info_template.json", "r") as f:
    dataset_info = json.load(f)

dataset_info["moduleName"] = task_name + "." + task_name
dataset_info["name"] = task_name
dataset_info["version"] = version
dataset_info["splits"][0]["shardLengths"] = [str(episodes_per_file)] * num_files 

with open(os.path.join(output_dir, "dataset_info.json"), "w") as f:
    json.dump(dataset_info, f, ensure_ascii=False, indent=2, sort_keys=True, separators=(',', ': '))

In [5]:
ret = shutil.copy("features_template.json", os.path.join(output_dir, "features.json"))
ret

'/media/sbr-tech/Desk SSD1/aloha/aloha_sim_multi_cucumber_dataset/aloha_sim_multi_cucumber_scripted_dataset/1.0.0/features.json'

In [6]:
sample_hdf5_file_1 = os.path.join(input_hdf5_dir_1, input_hdf5_basename.format(0))
sample_hdf5_file_2 = os.path.join(input_hdf5_dir_2, input_hdf5_basename.format(0))
sample_hdf5_file_3 = os.path.join(input_hdf5_dir_3, input_hdf5_basename.format(0))

In [7]:
NUM_EPISODE_1 = 600

IS_FIRSTS_1 = np.zeros(NUM_EPISODE_1, dtype=int)
IS_FIRSTS_1[0] = 1
DISCOUNTS_1 = np.ones(NUM_EPISODE_1, dtype=float)
IS_LASTS_1 = np.zeros(NUM_EPISODE_1, dtype=int)
IS_LASTS_1[-1] = 1
LANGUAGE_INSTRUCTION_1 = b"pick up one cucumber and put it in the bucket"
LANGUAGE_INSTRUCTIONS_1 = np.array([LANGUAGE_INSTRUCTION_1]*NUM_EPISODE_1)
REWARDS_1 = np.zeros(NUM_EPISODE_1, dtype=float)
REWARDS_1[-1] = 1
IS_TARMINALS_1 = np.zeros(NUM_EPISODE_1, dtype=int)
IS_TARMINALS_1[-1] = 1
METADATA_1 = sample_hdf5_file_1

In [8]:
NUM_EPISODE_2 = 600*2

IS_FIRSTS_2 = np.zeros(NUM_EPISODE_2, dtype=int)
IS_FIRSTS_2[0] = 1
DISCOUNTS_2 = np.ones(NUM_EPISODE_2, dtype=float)
IS_LASTS_2 = np.zeros(NUM_EPISODE_2, dtype=int)
IS_LASTS_2[-1] = 1
LANGUAGE_INSTRUCTION_2 = b"pick up two cucumbers and put them in the bucket"
LANGUAGE_INSTRUCTIONS_2 = np.array([LANGUAGE_INSTRUCTION_2]*NUM_EPISODE_2)
REWARDS_2 = np.zeros(NUM_EPISODE_2, dtype=float)
REWARDS_2[-1] = 1
IS_TARMINALS_2 = np.zeros(NUM_EPISODE_2, dtype=int)
IS_TARMINALS_2[-1] = 1
METADATA_2 = sample_hdf5_file_2

In [9]:
NUM_EPISODE_3 = 600*3

IS_FIRSTS_3 = np.zeros(NUM_EPISODE_3, dtype=int)
IS_FIRSTS_3[0] = 1
DISCOUNTS_3 = np.ones(NUM_EPISODE_3, dtype=float)
IS_LASTS_3 = np.zeros(NUM_EPISODE_3, dtype=int)
IS_LASTS_3[-1] = 1
LANGUAGE_INSTRUCTION_3 = b"pick up three cucumbers and put them in the bucket"
LANGUAGE_INSTRUCTIONS_3 = np.array([LANGUAGE_INSTRUCTION_3]*NUM_EPISODE_3)
REWARDS_3 = np.zeros(NUM_EPISODE_3, dtype=float)
REWARDS_3[-1] = 1
IS_TARMINALS_3 = np.zeros(NUM_EPISODE_3, dtype=int)
IS_TARMINALS_3[-1] = 1
METADATA_3 = sample_hdf5_file_3

In [10]:
def _image_bytes_feature(images):
    values = [tf.image.encode_jpeg(image).numpy() for image in images]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))

def _language_bytes_feature(langs):
    values = [lang.tobytes() for lang in langs]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))
    
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [11]:
def numpy_to_tf_example_1(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions):    
    # Featureを作成
    feature = {
        "steps/is_first": _int64_feature(IS_FIRSTS_1),
        "steps/action": _float_feature(actions.flatten()),
        "steps/discount": _float_feature(DISCOUNTS_1),
        "steps/is_last": _int64_feature(IS_LASTS_1),
        "steps/language_instruction": _language_bytes_feature(LANGUAGE_INSTRUCTIONS_1),
        "steps/rewards": _float_feature(REWARDS_1),
        "steps/observation/top": _image_bytes_feature(images_top),
        "steps/observation/angle": _image_bytes_feature(images_angle),
        "steps/observation/left_wrist": _image_bytes_feature(images_left_wrist),
        "steps/observation/right_wrist": _image_bytes_feature(images_right_wrist),
        "steps/is_terminal": _int64_feature(IS_TARMINALS_1),
        "steps/observation/state": _float_feature(states.flatten()),
        "episode_metadata/file_path": _bytes_feature(METADATA_1.encode())
    }
    
    features = tf.train.Features(feature=feature)
    example = tf.train.Example(features=features)
    
    return example


def numpy_to_tf_example_2(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions):    
    # Featureを作成
    feature = {
        "steps/is_first": _int64_feature(IS_FIRSTS_2),
        "steps/action": _float_feature(actions.flatten()),
        "steps/discount": _float_feature(DISCOUNTS_2),
        "steps/is_last": _int64_feature(IS_LASTS_2),
        "steps/language_instruction": _language_bytes_feature(LANGUAGE_INSTRUCTIONS_2),
        "steps/rewards": _float_feature(REWARDS_2),
        "steps/observation/top": _image_bytes_feature(images_top),
        "steps/observation/angle": _image_bytes_feature(images_angle),
        "steps/observation/left_wrist": _image_bytes_feature(images_left_wrist),
        "steps/observation/right_wrist": _image_bytes_feature(images_right_wrist),
        "steps/is_terminal": _int64_feature(IS_TARMINALS_2),
        "steps/observation/state": _float_feature(states.flatten()),
        "episode_metadata/file_path": _bytes_feature(METADATA_2.encode())
    }
    
    features = tf.train.Features(feature=feature)
    example = tf.train.Example(features=features)
    
    return example

def numpy_to_tf_example_3(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions):    
    # Featureを作成
    feature = {
        "steps/is_first": _int64_feature(IS_FIRSTS_3),
        "steps/action": _float_feature(actions.flatten()),
        "steps/discount": _float_feature(DISCOUNTS_3),
        "steps/is_last": _int64_feature(IS_LASTS_3),
        "steps/language_instruction": _language_bytes_feature(LANGUAGE_INSTRUCTIONS_3),
        "steps/rewards": _float_feature(REWARDS_3),
        "steps/observation/top": _image_bytes_feature(images_top),
        "steps/observation/angle": _image_bytes_feature(images_angle),
        "steps/observation/left_wrist": _image_bytes_feature(images_left_wrist),
        "steps/observation/right_wrist": _image_bytes_feature(images_right_wrist),
        "steps/is_terminal": _int64_feature(IS_TARMINALS_3),
        "steps/observation/state": _float_feature(states.flatten()),
        "episode_metadata/file_path": _bytes_feature(METADATA_3.encode())
    }
    
    features = tf.train.Features(feature=feature)
    example = tf.train.Example(features=features)
    
    return example

In [12]:
episode_idx_1 = 0
episode_idx_2 = 0
episode_idx_3 = 0

for file_idx in range(num_files):
    output_tfrecord_file = os.path.join(output_dir, output_basename.format(str(file_idx).zfill(5), str(num_files).zfill(5)))
    with tf.io.TFRecordWriter(output_tfrecord_file) as writer:

        hdf5_file_1 = os.path.join(input_hdf5_dir_1, input_hdf5_basename.format(episode_idx_1))
        with h5py.File(hdf5_file_1, 'r') as hdf5_f:
            states = hdf5_f["observations"]["qpos"][:].copy()
            images_top = hdf5_f["observations"]["images"]["top"][:].copy()
            images_angle = hdf5_f["observations"]["images"]["angle"][:].copy()
            images_left_wrist = hdf5_f["observations"]["images"]["left_wrist"][:].copy()
            images_right_wrist = hdf5_f["observations"]["images"]["right_wrist"][:].copy()
            actions = hdf5_f["action"][:].copy()
        example_1 = numpy_to_tf_example_1(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions)
        writer.write(example_1.SerializeToString())
        episode_idx_1 += 1

        for _ in range(2):
            hdf5_file_2 = os.path.join(input_hdf5_dir_2, input_hdf5_basename.format(episode_idx_2))
            with h5py.File(hdf5_file_2, 'r') as hdf5_f:
                states = hdf5_f["observations"]["qpos"][:].copy()
                images_top = hdf5_f["observations"]["images"]["top"][:].copy()
                images_angle = hdf5_f["observations"]["images"]["angle"][:].copy()
                images_left_wrist = hdf5_f["observations"]["images"]["left_wrist"][:].copy()
                images_right_wrist = hdf5_f["observations"]["images"]["right_wrist"][:].copy()
                actions = hdf5_f["action"][:].copy()
            example_2 = numpy_to_tf_example_2(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions)
            writer.write(example_2.SerializeToString())
            episode_idx_2 += 1

        for _ in range(3):
            hdf5_file_3 = os.path.join(input_hdf5_dir_3, input_hdf5_basename.format(episode_idx_3))
            with h5py.File(hdf5_file_3, 'r') as hdf5_f:
                states = hdf5_f["observations"]["qpos"][:].copy()
                images_top = hdf5_f["observations"]["images"]["top"][:].copy()
                images_angle = hdf5_f["observations"]["images"]["angle"][:].copy()
                images_left_wrist = hdf5_f["observations"]["images"]["left_wrist"][:].copy()
                images_right_wrist = hdf5_f["observations"]["images"]["right_wrist"][:].copy()
                actions = hdf5_f["action"][:].copy()
            example_3 = numpy_to_tf_example_3(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions)
            writer.write(example_3.SerializeToString())
            episode_idx_3 += 1

2024-08-12 13:22:47.968671: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-12 13:22:47.968925: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
